```c
/*!
 ************************************************************************
 * \brief
 *    Actually arithmetic encoding of one binary symbol by using
 *    the probability estimate of its associated context model
 ************************************************************************
 */
void biari_encode_symbol(EncodingEnvironmentPtr eep, signed short symbol, BiContextTypePtr bi_ct )
{
  register unsigned int range = Erange;
  register unsigned int low = Elow;
  // 1. 计算当前上下文的LPS概率
  // rLPS = 查表(bi_ct->state, range);
  unsigned int rLPS = rLPS_table_64x4[bi_ct->state][(range>>6) & 3];

  extern int cabac_encoding;

  if( cabac_encoding )
  {
    bi_ct->count++;
  }

  /* covers all cases where code does not bother to shift down symbol to be
   * either 0 or 1, e.g. in some cases for cbp, mb_Type etc the code symply
   * masks off the bit position and passes in the resulting value */
  // 2. 归一化符号 不是0就是1
  // if (symbol != 0) symbol = 1;
  if (symbol != 0)
    symbol = 1;
  // 3. 区间划分

  /*
  rLPS是查表得到的LPS（Least Probable Symbol，最不可能符号）对应的区间宽度。
  这一步将range减去rLPS，得到MPS（Most Probable Symbol，最可能符号）对应的区间宽度。

  作用：
  一部分宽度为range - rLPS，分配给MPS
  另一部分宽度为rLPS，分配给LPS

  |-------------------range-------------------|
  [low, low+range)
  |---------MPS--------|---LPS---|
  low                low+range-rLPS   low+range

  编码MPS：区间变为[low, low+range-rLPS)
  编码LPS：区间变为[low+range-rLPS, low+range)
  */
  range -= rLPS;
  // 小概率符号
  if (symbol != bi_ct->MPS)
  {
    low += range; // 把区间下界low向上移动到LPS区间的起点。
    range = rLPS; // 更新区间范围为LPS区间的宽度。
    // 概率state到达0了，代表MPS更换
    if (!bi_ct->state)
      bi_ct->MPS = bi_ct->MPS ^ 1;               // switch LPS if necessary
    bi_ct->state = AC_next_state_LPS_64[bi_ct->state]; // next lps state
  }
  else
    bi_ct->state = AC_next_state_MPS_64[bi_ct->state]; // next mps state

 /*
  1. 为什么用 bi_ct->state 代表概率？

  CABAC 的概率建模
  CABAC 并不直接存储概率值（如0.7、0.2），而是用有限状态机（FSM）来间接表示概率。
  bi_ct->state 就是这个有限状态机的状态编号，每个状态对应一个概率区间（通常是LPS的概率）。
  这样做的好处是：查表快、存储省、更新简单，而且可以避免浮点运算。

  具体实现
  state 的取值范围一般是0~63（或更大），每个状态对应一个LPS概率（通过查表rLPS_table_64x4获得）。
  状态越小，LPS概率越大（不确定性高）；状态越大，LPS概率越小（更确定）。

  例子
  | state | LPS概率（查表） | MPS概率 |
  |-------|----------------|---------|
  | 0 | 0.5 | 0.5 |
  | 32 | 0.1 | 0.9 |
  | 63 | 0.01 | 0.99 |

 */

  /* renormalisation 区间归一化 */
  /*

  1. 区间归一化的作用
    区间归一化就是在区间变得太小时，通过左移（放大）区间，并输出高位比特到码流，保证区间始终保持在一定的数值范围内，防止精度丢失。
  2. 区间归一化的过程
    触发条件：当前区间范围range < QUARTER
    1. 如果当前区间的下界low大于等于1/2（HALF），说明整个区间都在[0.5, 1)之间。此时可以确定最高位是1，可以输出1
    2. 如果当前区间的下界low小于1/4（QUARTER），说明整个区间都在[0, 0.25)之间。此时可以确定最高位是0，可以输出0。
    3. 如果区间在[0.25, 0.5)之间，最高位还不能确定。
    4. range区间放大
   */
  while (range < QUARTER)
  {
    if (low >= HALF)
    {
      put_one_bit_plus_outstanding(1);
      low -= HALF;
      // 输出1
    }
    else if (low < QUARTER) {
      /*
      put_one_bit_plus_outstanding 的作用
      该函数会输出一个比特（0或1），并把之前“待输出的跟随比特Ebits_to_follow”全部输出为相反的值。
      例如，当前输出1，之前有3个“待输出”，就会输出1，然后输出3个0（即1后面跟3个0）。
      这样保证了比特流的正确性和算术编码的精度。
      */
      put_one_bit_plus_outstanding(0);
      // 输出0
    }
    else {
      //
        Ebits_to_follow++;
        low -= QUARTER;
    }
    // 区间放大
    low <<= 1;
    range <<= 1;
  }
  Erange = range;
  Elow = low;
  eep->C++;

}
```